# Project 02 - Due Friday, November 11 at 12pm

*Objectives*: Gain experience loading a transactional dataset into a relational database model you define yourself and using SQL to explore its contents. Transform the data into a star schema, documenting the schema visually, and explore the transformed data analytically by writing and executing a number of SQL queries using common syntax and functions and describing your findings.  Gain practice working on these tasks with a partner.

*Grading criteria*: The tasks should all be completed, and questions should all be answered with SQL queries, diagrams, and charts in the space provided, unless a text answer is requested.  The notebook itself should be completely reproducible at datanotebook.org, from start to finish: another person should be able to use the same code to obtain the same results as yours.  Note that you will receive no more than partial credit if you do not add text/markdown cells explaining your thinking at each major step in each problem.

*Attestation*: **Work in pairs**.  At the end of your submitted notebook, identify the work each partner performed and attest that each contributed substantially to the work.

*Deadline*: Friday, November 11, 12pm.  One member of each pair must submit your notebook to Blackboard; you should not both submit it separately.  You may each push a copy to your own GitHub repository.


## Setup - select and obtain data

The US City Open Data Census has a variety of [transactional data from all over the country](http://us-city.census.okfn.org/).  Select one topic and one city from this Census, download the data, and explain your reason for this choice before proceeding.  Provide a link to the web page for the data set you've chosen.

I strongly encourage you to select a dataset from among the following topics:  Crime, Property Assessment, Campaign Finance Contributions, Service Requests (311), and Web Analytics.

By *transactional* data I mean records at the grain of one event per business process.  In the case of Service Requests (311), for example, that would require data at the level of each individual report of a service request.  If this isn't clear, think about the bike trip data - each individual ride was included - and look for data at that level of event/process specificity.  Avoid data like the Connecticut boating data, which was summarized by year, not individual transactions.

Please aim for a dataset of at least 10,000 individual records, but less than 250,000 records.  A little more or a little less is okay, but try to stay within these limits.

In [1]:
!wget https://data.baltimorecity.gov/api/views/wsfq-mvij/rows.csv

--2016-11-04 23:41:50--  https://data.baltimorecity.gov/api/views/wsfq-mvij/rows.csv
Resolving data.baltimorecity.gov (data.baltimorecity.gov)... 52.206.140.205
Connecting to data.baltimorecity.gov (data.baltimorecity.gov)|52.206.140.205|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: unspecified [text/csv]
Saving to: ‘rows.csv’

rows.csv                [   <=>                ]  32.92M  70.8MB/s   in 0.5s   

Last-modified header invalid -- time-stamp ignored.
2016-11-04 23:41:50 (70.8 MB/s) - ‘rows.csv’ saved [34515653]



## Problem 1 - examine the data (20 points)

Use `csvstat`, `pandas`, or other tools as you see fit and observe the contents of your dataset.  Review important points from any metadata provided.  Describe what you see:  What do the columns mean?  Are there null values?  Which columns interest you the most?  Which columns present some opportunities, challenges, or questions you would like to ask?

In [2]:
!wc -l rows.csv

283615 rows.csv


In [3]:
!shuf -n 10000 rows.csv | csvstat

  1. 01/21/2015
	<class 'datetime.date'>
	Nulls: False
	Min: 2011-01-01
	Max: 2016-10-29
	Unique values: 2102
	5 most frequent values:
		2013-11-18:	13
		2013-08-31:	13
		2014-08-29:	12
		2013-07-07:	12
		2013-05-22:	12
  2. 23:40:00
	<class 'str'>
	Nulls: False
	Unique values: 1278
	5 most frequent values:
		20:00:00:	229
		18:00:00:	227
		19:00:00:	223
		22:00:00:	223
		16:00:00:	212
	Max length: 8
  3. 4E
	<class 'str'>
	Nulls: False
	Unique values: 69
	5 most frequent values:
		4E:	1697
		6D:	1336
		5A:	958
		7A:	900
		6G:	545
	Max length: 4
  4. 2100 WASHINGTON BLVD
	<class 'str'>
	Nulls: True
	Unique values: 6224
	5 most frequent values:
		1200 W PRATT ST:	24
		300 LIGHT ST:	23
		200 E PRATT ST:	20
		1500 RUSSELL ST:	20
		3200 TIOGA PW:	17
	Max length: 37
  5. COMMON ASSAULT
	<class 'str'>
	Nulls: False
	Unique values: 15
	5 most frequent values:
		LARCENY:	2156
		COMMON ASSAULT:	1697
		BURGLARY:	1564
		LARCENY FROM AUTO:	1336
		AGG. ASSAULT:	975
	Max length: 20
  6. O
	<class 's

In [4]:
!csvcut -n rows.csv

  1: CrimeDate
  2: CrimeTime
  3: CrimeCode
  4: Location
  5: Description
  6: Inside/Outside
  7: Weapon
  8: Post
  9: District
 10: Neighborhood
 11: Location 1
 12: Total Incidents


## Problem 2 - define a database model, load the data, and explore (20 points)

Based on what you found above, create and connect to a new database, define a database table in it, and load this dataset into it.  You may use either of the methods for this step you have seen in earlier class notebooks.  You may choose to eliminate variables/columns if they are not relevant or interesting to you - explain your reasoning if you do.  Either way, you should load a majority of the columns present in the source dataset as it is in its raw form into the database, and all of its rows.

Once your data has loaded successfully, run a `COUNT(*)` query to verify that all the data has loaded correctly.

Explore the data to zero in on a few themes you would like to further study analytically.  Discuss columns that present opportunities for extraction into dimensions, and identify the specific columns that contain facts you want to measure.

In [5]:
%load_ext sql
!echo 'redspot' | sudo -S service postgresql restart
!createdb -U dbuser BaltimoreCrime
%sql postgresql://dbuser@localhost:5432/BaltimoreCrime

/opt/conda/lib/python3.5/site-packages/IPython/config.py:13: ShimWarning: The `IPython.config` package has been deprecated. You should import from traitlets.config instead.
  "You should import from traitlets.config instead.", ShimWarning)
/opt/conda/lib/python3.5/site-packages/IPython/utils/traitlets.py:5: UserWarning: IPython.utils.traitlets has moved to a top-level traitlets package.
  warn("IPython.utils.traitlets has moved to a top-level traitlets package.")


[sudo] password for jovyan: Restarting PostgreSQL 9.5 database server: main.


'Connected: dbuser@BaltimoreCrime'

In [7]:
%%sql
DROP TABLE IF EXISTS BaltimoreCrime;
CREATE TABLE BaltimoreCrime (
    CrimeDate TIMESTAMP,
    CrimeTime VARCHAR(64),
    CrimeCode VARCHAR(64),
    Location VARCHAR(64),
    Description VARCHAR(64),
    Inside_Outside VARCHAR(64),
    Weapon VARCHAR(64),
    Post INTEGER,
    District VARCHAR(64),
    Neighborhood VARCHAR(64),
    Location_1 VARCHAR(64),
    Total_Incidents INTEGER
)

Done.
Done.


[]

In [8]:
%%sql
COPY BaltimoreCrime FROM '/home/jovyan/work/rows.csv'
CSV
HEADER
QUOTE '"'
DELIMITER ',';

283614 rows affected.


[]

We used COUNT() to verify that the data was all loaded into the table.

In [10]:
%%sql
SELECT COUNT(*) FROM BaltimoreCrime;

1 rows affected.


count
283614


We checked the first five rows to see that the data formats carried over properly.

In [13]:
%%sql
SELECT *
FROM BaltimoreCrime
LIMIT 5;

5 rows affected.


crimedate,crimetime,crimecode,location,description,inside_outside,weapon,post,district,neighborhood,location_1,total_incidents
2016-10-29 00:00:00,00:01:00,6E,4700 FURLEY AVE,LARCENY,O,None,443,NORTHEASTERN,Frankford,"(39.3285900000, -76.5517800000)",1
2016-10-29 00:00:00,00:01:00,7A,3500 LIBERTY HEIGHTS AVE,AUTO THEFT,O,None,642,NORTHWESTERN,Forest Park,"(39.3250400000, -76.6748500000)",1
2016-10-29 00:00:00,00:58:00,6C,0 LIGHT ST,LARCENY,I,None,111,CENTRAL,Downtown,"(39.2884300000, -76.6137200000)",1
2016-10-29 00:00:00,01:00:00,6D,2200 BERLIN ST,LARCENY FROM AUTO,O,None,921,SOUTHERN,Saint Paul,"(39.2695600000, -76.6447300000)",1
2016-10-29 00:00:00,01:00:00,7A,2800 E FEDERAL ST,AUTO THEFT,O,None,332,EASTERN,Berea,"(39.3091700000, -76.5780900000)",1


To make the data easier to explore, we separated the data column into year, month, and day.

In [15]:
%%sql
ALTER TABLE BaltimoreCrime
ADD COLUMN year NUMERIC;

Done.


[]

In [16]:
%%sql
ALTER TABLE BaltimoreCrime
ADD COLUMN month NUMERIC;

Done.


[]

In [17]:
%%sql
ALTER TABLE BaltimoreCrime
ADD COLUMN day NUMERIC;

Done.


[]

In [18]:
%%sql
UPDATE BaltimoreCrime
SET year = EXTRACT(YEAR FROM crimedate), 
    month = EXTRACT(MONTH FROM crimedate),
    day = EXTRACT(DAY FROM crimedate);

283614 rows affected.


[]

The following cell lets us test that the year, month, and day columns were created and filled correctly.

In [19]:
%%sql
SELECT *
FROM BaltimoreCrime
LIMIT 5;

5 rows affected.


crimedate,crimetime,crimecode,location,description,inside_outside,weapon,post,district,neighborhood,location_1,total_incidents,year,month,day
2016-10-28 00:00:00,11:26:00,3CF,None,ROBBERY - COMMERCIAL,I,FIREARM,731,WESTERN,None,None,1,2016,10,28
2016-10-27 00:00:00,03:15:00,4F,None,ASSAULT BY THREAT,I,None,614,NORTHWESTERN,None,None,1,2016,10,27
2016-08-17 00:00:00,17:00:00,6E,None,LARCENY,O,None,332,EASTERN,None,None,1,2016,8,17
2016-10-26 00:00:00,16:40:00,6C,5300 YORK RD,LARCENY,I,None,523,NORTHERN,Homeland,"(39.3550800000, -76.6099100000)",1,2016,10,26
2015-12-18 00:00:00,15:00:00,3B,None,ROBBERY - STREET,O,None,224,SOUTHEASTERN,None,None,1,2015,12,18


## Problem 3 - define a star schema, and load your data into it (30 points)

Define a star schema consisting of at least one fact table and at least two dimensions.  Transform your source data into the new fact table and dimensions.  Discuss your key modeling decisions.

Document your star schema with a diagram.  There several tools available for this, such as Google Docs and Microsoft Visio.

## Problem 4 - explore your data in its new schema (30 points)

Strictly using the dimensional model tables you defined and populated for Problem 3, explore your data in its new form.  Use any of the query strategies we've seen in class, including transformations, aggregates, subqueries, rollups, and, of course, joins.  Add plots to highlight particular themes that stand out.  Describe your thinking and observations along the way.

## Bonus (10 points for one of A or B)


### Option A - Automating ETL

Consider the work you did to load your original raw dataset and then transform it into a dimensional model for analysis.  What would it take to automate this process?  How often would you need to update the data?  Could you easily automate any data cleaning steps?  What checks would you need to put in place to ensure quality?  Would humans need to be involved, or could you automate it all?

Discuss.


### Option B - Augmenting dimensions

Considering your dimensional model, what external data could you find to augment it?  As in the case of bike trips, where weather might provide an interesting added dimension/context not present in the original data, you can probably find another source of data to complement your own model.  Identify one such source and add it to your model, demonstrating its value with a few new queries.